# Data loading and modeling choices

Our target are `prezzo_min` and `prezzo_max`. We keep `affitto_min` and `affitto_max` in our model, even though in a real world setting we might have to forecast _all four_ of these values. We keep the rental prices in the model to have some quantitative variables in our training set. We drop the observations where the target variable is missing.

In [1]:
from omi.dataloader import ValuesDataLoader

target_cols = ["prezzo_min", "prezzo_max"]

values_dataloader = ValuesDataLoader(year=2022)
values = (
    values_dataloader
    .load(semester=2)
    .query("prezzo_min > 0 and prezzo_max > 0")
)

data, target = values.drop(columns=target_cols), values.filter(target_cols)

In [2]:
data.head()

,area_territoriale,regione,provincia,comune_codice_istat,comune_codice_catastale,comune_codice_nazionale,comune_denominazione,fascia,zona,linkzona,tipologia_codice,tipologia_descrizione,condizione,affitto_min,affitto_max,affitto_superficie
0,NORD-OVEST,PIEMONTE,AL,1006003.0,A2AA,A182,ALESSANDRIA,B,B1,AL00000001,20,Abitazioni civili,NORMALE,4.0,6.0,L
1,NORD-OVEST,PIEMONTE,AL,1006003.0,A2AA,A182,ALESSANDRIA,B,B1,AL00000001,13,Box,NORMALE,4.0,5.8,L
2,NORD-OVEST,PIEMONTE,AL,1006003.0,A2AA,A182,ALESSANDRIA,B,B1,AL00000001,14,Posti auto coperti,NORMALE,2.7,3.8,L
3,NORD-OVEST,PIEMONTE,AL,1006003.0,A2AA,A182,ALESSANDRIA,B,B1,AL00000001,15,Posti auto scoperti,NORMALE,2.5,3.3,L
4,NORD-OVEST,PIEMONTE,AL,1006003.0,A2AA,A182,ALESSANDRIA,B,B1,AL00000001,9,Magazzini,NORMALE,4.6,5.4,L


# Metrics

In absence of business requirements or KPIs, the metric of choice will be the mean absolute error (MAE), which denotes the error in euro/squared meter.

# Feature selection

In [3]:
from sklearn import model_selection
from sklearn import compose, preprocessing, pipeline
from sklearn import linear_model, ensemble, multioutput
from sklearn import metrics

In [4]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(data, target, train_size=0.8)

We only keep categorical and numerical columns, and not string columns like `comune_codice_istat`, to avoid the curse of dimensionality: once encoded, they would have too many levels. This is not an issue for ensembles of trees, but I chose this approach for the following reasons:

1. Faster convergence
2. More dense data
3. Less parameters to interpret
4. Provide a baseline (e.g. in the future we could regress against the municipalities and see the improvement).

Ideally, one could encode these variables. Bigger cities could be denoted by an indicator variable (e.g. Milan); clustering algorithms with external data (e.g. GDP) could be used. Nevertheless, an indicator for each municipality is too much: my hope/intuition is that there are already enough categorical values to capture the "core" variance across cities (e.g. capture why an apartment is more expensive in Milan than a rural village in the midst of Molise).

A couple more notes. We might also want to consider dropping either the `area_territoriale` or `regione` , since we have categorical variables available at the `provincia` and municipal level. Besides, we could lump some levels in other categories, too, such as those occurring the least in `tipologia_descrizione`.

In [5]:
numeric_selector = compose.make_column_selector(dtype_include="number")
categorical_selector = compose.make_column_selector(dtype_include="category")

# Baseline

In [6]:
ordinal_encoder = preprocessing.OrdinalEncoder(handle_unknown="use_encoded_value", unknown_value=-1)

gbrt_transformer = compose.ColumnTransformer([
    ("ordinal_encoding", ordinal_encoder, categorical_selector),
    ("quantitative_cols", "passthrough", numeric_selector)
],
    remainder="drop"
)

gbrt_pipeline = pipeline.Pipeline([
    ("feature_eng", gbrt_transformer),
    ("gbft", multioutput.MultiOutputRegressor(ensemble.HistGradientBoostingRegressor(random_state=42)))
])

gbrt_pipeline

Pipeline(steps=[('feature_eng',
                 ColumnTransformer(transformers=[('ordinal_encoding',
                                                  OrdinalEncoder(handle_unknown='use_encoded_value',
                                                                 unknown_value=-1),
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x174b690c0>),
                                                 ('quantitative_cols',
                                                  'passthrough',
                                                  <sklearn.compose._column_transformer.make_column_selector object at 0x174b6a680>)])),
                ('gbft',
                 MultiOutputRegressor(estimator=HistGradientBoostingRegressor(random_state=42)))])

Given the high number of categories and the tradeoffs in category encoding, we choose as our baseline a XGBoost-like ensemble model (GBRT, or Gradient Boosted Regression Tree). These algorithms are known to give the best results with tabular data and will capture non-linear relationships. The GBRT is also robust to outliers and scale, so we do need to perform any feature engineering for quantitative variables. Finally, the GBRT accepts `NaN` values. Given that `affitto_min` and `affitto_max` contain some null values, this spares us some more feature engineering.

In [7]:
_ = gbrt_pipeline.fit(X_train, y_train)

In [8]:
gbrt_pipeline["gbft"].estimator.scoring

'loss'

In [9]:
cv_results = model_selection.cross_validate(
    estimator=gbrt_pipeline,
    X=X_train, y=y_train,
    scoring="neg_mean_absolute_error"
)

Here the folds are randomly sampled; given some domain knowledge, we might stratify with respect to a relevant feature.

In [10]:
import pandas as pd

pd.DataFrame(cv_results)

,fit_time,score_time,test_score
0,3.036252,0.096752,-113.163844
1,4.035801,0.085758,-113.761199
2,3.150641,0.130087,-115.281308
3,3.011556,0.127260,-113.206757
4,2.528949,0.083819,-118.059284


The average error is 1e2 euro/squared meter - somewhat high considering that most houses sell at around 1e3 euro/squared meter. On the other hand, we have to keep in mind that basically no feature engineering occurred, and no hyperparameters were tuned.

If we do not average the MAE but compute it separately for each target, we see that it is lower for the minimum threshold (though it is still the same order of magnitude).

In [11]:
def score(model, X_test, y_test):
    y_pred = model.predict(X_test)
    return metrics.mean_absolute_error(y_test, y_pred, multioutput="raw_values")

mae = score(gbrt_pipeline, X_test, y_test)

mae

array([ 97.22313988, 134.38706504])